In [32]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [23]:
df = pd.read_csv('/Users/augustincablant/Desktop/challenge_SNCF/train_f_x.csv')
y = pd.read_csv('/Users/augustincablant/Desktop/challenge_SNCF/y_train_sncf.csv')
def transform_y(dataset):
    test = dataset
    test['date'] = 0 
    test['station'] = 0

    for i in test.index.to_list():
        row = test['index'][i]
        test['date'][i] = row.split('_')[0]
        test['station'][i] = row.split('_')[1]
    return test

y = transform_y(y)
df_train = pd.merge(df, y, how = 'left', on = ['date', 'station'])
df_train['date'] = pd.to_datetime(df_train['date'])

df_train['jour'] = 0 
df_train['mois'] = 0 
df_train['année'] = 0 
for i in df_train.index:
    date = df_train['date'][i]
    df_train['jour'][i] = date.day
    df_train['mois'][i] = date.month 
    df_train['année'][i] = date.year

df_train.head()

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/4028105344.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'][i] = row.split('_')[0]
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/4028105344.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['station'][i] = row.split('_')[1]
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/4028105344.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,date,station,job,ferie,vacances,index,y,jour,mois,année
0,2015-01-01,1J7,1,1,1,2015-01-01_1J7,7,1,1,2015
1,2015-01-01,O2O,1,1,1,2015-01-01_O2O,0,1,1,2015
2,2015-01-01,8QR,1,1,1,2015-01-01_8QR,9,1,1,2015
3,2015-01-01,UMC,1,1,1,2015-01-01_UMC,9,1,1,2015
4,2015-01-01,FK3,1,1,1,2015-01-01_FK3,28,1,1,2015


In [44]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    zero_indices = y_true == 0
    non_zero_error = np.abs((y_true[~zero_indices] - y_pred[~zero_indices]) / y_true[~zero_indices])
    return np.mean(non_zero_error) * 100

In [34]:
label_encoder = LabelEncoder()
df_train['station_num'] = label_encoder.fit_transform(df_train['station'])
df_train.head()

,date,station,job,ferie,vacances,index,y,jour,mois,année,station_num
0,2015-01-01,1J7,1,1,1,2015-01-01_1J7,7,1,1,2015,24
1,2015-01-01,O2O,1,1,1,2015-01-01_O2O,0,1,1,2015,295
2,2015-01-01,8QR,1,1,1,2015-01-01_8QR,9,1,1,2015,107
3,2015-01-01,UMC,1,1,1,2015-01-01_UMC,9,1,1,2015,367
4,2015-01-01,FK3,1,1,1,2015-01-01_FK3,28,1,1,2015,199


In [35]:
X = df_train[['jour', 'mois', 'année', 'station_num', 'job', 'ferie', 'vacances']]
Y = df_train['y']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)

MAPE is :  nan


/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/4111968303.py:3: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/4111968303.py:3: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [45]:
print("MAPE is : ",  mean_absolute_percentage_error(y_test.values, predictions))

MAPE is :  127.03759224463656


In [46]:
y_exemple = pd.read_csv('/Users/augustincablant/Desktop/y_exemple_sncf_d9so9pm.csv')
y_exemple.head()

,index,y
0,2023-01-01_1J7,10000
1,2023-01-01_O2O,10000
2,2023-01-01_8QR,10000
3,2023-01-01_L58,10000
4,2023-01-01_UMC,10000


In [52]:
X_predict = pd.read_csv('/Users/augustincablant/Desktop/challenge_SNCF/test_f_x_THurtzP.csv')
X_predict['station_num'] = label_encoder.transform(X_predict['station'])
X_predict['date'] = pd.to_datetime(X_predict['date'])
X_predict['jour'] = 0 
X_predict['mois'] = 0 
X_predict['année'] = 0 
for i in X_predict.index:
    date = X_predict['date'][i]
    X_predict['jour'][i] = date.day
    X_predict['mois'][i] = date.month 
    X_predict['année'][i] = date.year

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/314973587.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_predict['jour'][i] = date.day
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/314973587.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_predict['mois'][i] = date.month
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_20137/314973587.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [54]:
predictions = rf_model.predict(X_predict[['jour', 'mois', 'année', 'station_num', 'job', 'ferie', 'vacances']])
predictions

array([ 27.8 ,  16.85,  22.53, ...,  55.89, 550.37, 209.08])

In [64]:
X_predict['y'] = predictions
y_pred = X_predict[['index', 'y']]
y_pred.set_index('index', inplace = True)
y_pred.to_csv('/Users/augustincablant/Desktop/challenge_SNCF/output1.csv')